In [ ]:
import os
import subprocess
from subprocess import CalledProcessError

EXE_DIR_PATH = "../build/src/exe/"
WORK_DIR = "../ir_demo/"


def run_cmd(cmd):
    try:        
        print("$ ", cmd)
        print()
        output = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT)
        print(output.decode('utf-8'))
    except CalledProcessError as e:
        print(e)
        print("output: {}".format(e.output.decode('utf-8')))
        

cmd = "ls {EXE_DIR_PATH}".format( \
        EXE_DIR_PATH=EXE_DIR_PATH)
run_cmd(cmd)

# Image Retrieval

image retrieval system is already implemented. 

We need input for the system, and they are:

(requirements)
* project_dir
* database
* vocab_tree
* database_image_list
* query_image_list

(optional)
* num_images
* num_verifications
* max_num_features


# Vocabulary

You can download pre-trained vocab tree from here: https://demuc.de/colmap/#download

Put the file into WORK_DIR

In [ ]:
VOCAB_TREE_PATH = os.path.join(WORK_DIR, "vocab_tree-262144.bin")

## Alternatively, you can build it from scratch

use `vocab_tree_builder`

In [ ]:
# TODO: run voacb_tree_builder command with appropriate args

# Database

To keep intermediate data for image retrieval, we use SQLite3 database. 

In [ ]:
DB_PATH = os.path.join(WORK_DIR, "database")

cmd = "{EXE_DIR_PATH}database_creator --database_path {DB_PATH}".format( \
                EXE_DIR_PATH=EXE_DIR_PATH, \
                WORK_DIR=WORK_DIR, \
                DB_PATH=DB_PATH)
run_cmd(cmd)

In [ ]:
cmd = "{EXE_DIR_PATH}vocab_tree_retriever -h --database_path {DB_PATH} --vocab_tree_path {VOCAB_TREE_PATH}".format( \
                EXE_DIR_PATH=EXE_DIR_PATH, \
                DB_PATH=DB_PATH, \
                VOCAB_TREE_PATH=VOCAB_TREE_PATH)
run_cmd(cmd)